# EVALUAR Y GENERAR REPORTS PARA LOS ALUMNOS

### CONSIDERACIONES PARA QUE FUNCIONE:

- HAY QUE INSTALAR LIBREOFFICE EN EL TERMINAL

        -  sudo apt update

        - sudo apt install libreoffice

- EL TEMPLATE DEL REPORT PARA EL PONENTE TIENE QUE ESTAR EN:

        - TEMPLATE_PATH = '/workspace/template/Evaluacion_Sesion_template.docx'

- FICHERO RESPUESTAS:


        - Eliminar las primeras 7 columnas del DataFrame, asumiendo que no son necesarias para el análisis.

        - Tiene que tener 8 preguntas numéricas de 0 a 5 y 4 con texto libre

        - Las preguntas 11 y 12 son las que hay que usar pare evaluar


In [1]:
%pip install docxtpl

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install fpdf2

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from datetime import datetime
from docxtpl import DocxTemplate
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import os
import subprocess
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import smtplib
import concurrent.futures
import os
from dotenv import load_dotenv
from tqdm import tqdm
from openai import OpenAI
from datetime import datetime, timedelta
from fpdf import FPDF
from fpdf.enums import XPos, YPos


## DEFINICIÓN DE LAS FUNCIONES

In [4]:
def leer_criterios(CRITERIOS_PATH):
    """
    Lee el archivo de criterios y separa los criterios para Q11_Específica 1 y Q12_Específica 2.
    
    Parámetros:
    - CRITERIOS_PATH: Ruta del archivo de texto que contiene los criterios.

    Retorna:
    - Una tupla con dos listas: la primera contiene los criterios de Q11 y la segunda los de Q12.

    Manejo de errores:
    - Verifica si el archivo existe y puede ser leído.
    - Verifica que el archivo contenga las secciones esperadas para Q11 y Q12.
    """

    criterios_q11 = []
    criterios_q12 = []
    leyendo_q11 = False
    leyendo_q12 = False

    try:
        # Intentar abrir el archivo
        with open(CRITERIOS_PATH, 'r') as file:
            contenido = file.read()

        # Procesar el contenido línea por línea
        for linea in contenido.splitlines():
            if "Criterios para Q11_Especifica1:" in linea:
                leyendo_q11 = True
                leyendo_q12 = False
            elif "Criterios para Q12_Especifica2:" in linea:
                leyendo_q11 = False
                leyendo_q12 = True
            elif leyendo_q11 and linea.strip():
                criterios_q11.append(linea.strip())
            elif leyendo_q12 and linea.strip():
                criterios_q12.append(linea.strip())
        
        # Verificar si se encontraron criterios en ambas secciones
        if not criterios_q11:
            raise ValueError("No se encontraron criterios para Q11_Session content review 1 en el archivo.")
        if not criterios_q12:
            raise ValueError("No se encontraron criterios para Q12_Específica 2 en el archivo.")

        # Imprimir mensaje de éxito con el número de criterios leídos para Q11 y Q12
        print(f"Se leyeron {len(criterios_q11)} criterios para Q11 y {len(criterios_q12)} criterios para Q12 del archivo '{CRITERIOS_PATH}'.")

    except FileNotFoundError:
        print(f"Error: El archivo '{CRITERIOS_PATH}' no se encontró. Verifica la ruta.")
    except IOError:
        print(f"Error: No se pudo leer el archivo '{CRITERIOS_PATH}'. Verifica los permisos.")
    except ValueError as e:
        print(f"Error: {e}")

    return criterios_q11, criterios_q12


In [5]:
def leer_encuestas(file_path_csv):
    """
    Carga un archivo CSV en un DataFrame, verifica si está vacío y selecciona
    columnas específicas para su posterior análisis.

    Parámetros:
    - file_path_csv: Ruta al archivo CSV que contiene los datos de evaluación.

    Retorna:
    - df: DataFrame con las columnas seleccionadas ('Nombre completo', 'Nombre de usuario', 'Q11_Específica 1', 'Q12_Específica 2').

    Excepciones:
    - ValueError: Si el archivo CSV está vacío o no contiene datos válidos.
    """
    
    # Verificar si el archivo CSV existe
    if not os.path.exists(file_path_csv):
        raise FileNotFoundError(f"El archivo CSV {file_path_csv} no existe.")
    
    # Leer los datos del archivo CSV en un DataFrame
    try:
        df = pd.read_csv(file_path_csv)
    except Exception as e:
        print(f"Error al leer el archivo CSV: {e}")
        return None

    # Verificar si el DataFrame está vacío
    if df.empty:
        raise ValueError("El archivo CSV está vacío o no contiene datos válidos.")

    # Definir la lista de columnas que queremos seleccionar
    columnas_seleccionadas = ['Nombre completo', 'Nombre de usuario', 'Q11_Session content review 1', 'Q12_Session content review 2']

    # Verificar si las columnas están presentes
    columnas_disponibles = all(col in df.columns for col in columnas_seleccionadas)
    if not columnas_disponibles:
        raise ValueError(f"El archivo CSV no contiene las columnas necesarias: {columnas_seleccionadas}")

    # Seleccionar las columnas específicas del DataFrame
    df = df[columnas_seleccionadas]

    # Verificar filas procesadas correctamente
    filas_procesadas = df.shape[0]
    
    # Imprimir mensaje de éxito con el número de filas leídas
    print(f"Se han leído con éxito {filas_procesadas} registros del archivo CSV.")

    # Retornar el DataFrame filtrado
    return df


In [6]:
def evaluar_respuesta_gpt(respuesta, criterios, pregunta_1, pregunta_2):
    """
    Utiliza la API de OpenAI para evaluar la respuesta del estudiante basándose en los criterios.
    
    Parámetros:
    - respuesta: Respuesta del estudiante.
    - criterios: Criterios correspondientes para evaluar la respuesta.
    - pregunta_1: La primera pregunta para verificar si la respuesta es solo una reformulación.
    - pregunta_2: La segunda pregunta para verificar si la respuesta es solo una reformulación.

    Retorna una nota y una explicación de la evaluación.
    """

    # Crear el prompt con la respuesta, las preguntas y los criterios, especificando un formato estructurado
    prompt = f"""
        Evaluate the following answer against these criteria, considering that the answer can be in English or Spanish:

        Criteria:
        {criterios}

        Student response:
        {respuesta}

        Questions:
        - {pregunta_1}
        - {pregunta_2}

        Important:
        - If the answer is only a repetition or rephrasing of any of the questions, the mark should be 0.
        - Assess the accuracy, coherence, relevance and clarity of the answer, regardless of whether it is written in English or Spanish.
        - Do not penalise minor spelling or grammatical errors, unless they affect the understanding of the concept.
        - Consider the student's ability to be concise but thorough. The answer should cover the key points without rambling.
        - If the content is confusing, incomplete or unrelated to the criteria, apply penalties accordingly.
        - If the answer is not related to the questions or criteria, the mark should be 0.

        Provide the assessment in the following EXACT format:
        Nota: <numeric value between 1 and 10 to one decimal place, or 0 if the answer is a rephrasing of the question>
        Comentario: <detailed explanation in English of the evaluation, indicating strengths, areas for improvement and any conceptual errors>
    """


    try:
        # Inicializar cliente OpenAI
        cliente = OpenAI()

        # Crear la solicitud de completado del chat
        response = cliente.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Eres un evaluador que analiza respuestas breves de estudiantes basadas en criterios específicos."},
                {"role": "user", "content": prompt}
            ]
        )

        # Obtener la respuesta generada por el modelo
        evaluacion = response.choices[0].message.content

        # Procesar la evaluación para extraer la nota y el comentario
        try:
            # Dividir la evaluación en líneas y buscar la nota y el comentario
            lines = evaluacion.split("\n")

            # Buscar la línea que contiene la nota y procesarla
            nota_line = next(line for line in lines if line.startswith("Nota:"))
            nota = nota_line.split(":")[1].strip()  # Extraer la nota después de 'Nota:'

            # Buscar la línea que contiene el comentario
            comentario_line = next(line for line in lines if line.startswith("Comentario:"))
            comentario = comentario_line.split(":", 1)[1].strip()  # Extraer el comentario después de 'Comentario:'

        except Exception as e:
            print(f"Error al procesar la evaluación: {e}")
            return None, "Error al procesar la evaluación."

        return nota, comentario

    except Exception as e:
        print(f"Error al evaluar la respuesta: {e}")
        return None, "Error en la evaluación."



In [7]:

def procesar_respuesta(index, row, criterios_q11, criterios_q12):
    """
    Procesa las respuestas de una fila del DataFrame.
    Evalúa las respuestas de Q11 y Q12 y retorna los resultados.
    """
    # Obtener las respuestas para Q11 y Q12 de las columnas correspondientes
    respuesta_q11 = row['Q11_Session content review 1']
    respuesta_q12 = row['Q12_Session content review 2']

    # Evaluar la respuesta a Q11
    nota_q11, comentario_q11 = evaluar_respuesta_gpt(respuesta_q11, "\n".join(criterios_q11), PREGUNTA_1, PREGUNTA_2)
    
    # Evaluar la respuesta a Q12
    nota_q12, comentario_q12 = evaluar_respuesta_gpt(respuesta_q12, "\n".join(criterios_q12), PREGUNTA_1, PREGUNTA_2)

    return index, nota_q11, comentario_q11, nota_q12, comentario_q12

In [8]:
def evaluar_respuestas_df_paralelo(df, criterios_q11, criterios_q12):
    """
    Recorre el DataFrame df en paralelo, evalúa las respuestas de Q11 y Q12 
    usando la función evaluar_respuesta_gpt, y crea nuevas columnas para almacenar 
    las notas y los comentarios para cada respuesta.
    """
    
    # Inicializar las nuevas columnas en el DataFrame
    df['Nota Q11'] = None
    df['Comentario Q11'] = None
    df['Nota Q12'] = None
    df['Comentario Q12'] = None

    filas_procesadas = 0  # Contador de filas procesadas correctamente

    # Usar ThreadPoolExecutor para ejecutar en paralelo
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Enviar todas las filas al pool de threads para procesamiento
        futuros = {
            executor.submit(procesar_respuesta, index, row, criterios_q11, criterios_q12): index
            for index, row in df.iterrows()
        }

        # Recoger los resultados conforme se completan
        for future in concurrent.futures.as_completed(futuros):
            try:
                index, nota_q11, comentario_q11, nota_q12, comentario_q12 = future.result()

                # Guardar los resultados en las nuevas columnas del DataFrame
                df.at[index, 'Nota Q11'] = nota_q11
                df.at[index, 'Comentario Q11'] = comentario_q11
                df.at[index, 'Nota Q12'] = nota_q12
                df.at[index, 'Comentario Q12'] = comentario_q12

                filas_procesadas += 1  # Incrementar el contador de filas procesadas correctamente
            except Exception as e:
                print(f"Error al procesar la fila {index}: {e}")

    # Convertir las columnas Nota Q11 y Nota Q12 a float de forma segura
    df['Nota Q11'] = pd.to_numeric(df['Nota Q11'], errors='coerce')
    df['Nota Q12'] = pd.to_numeric(df['Nota Q12'], errors='coerce')

    # Calcular la nota final como la media de Nota Q11 y Nota Q12
    df['Nota Final'] = (df['Nota Q11'] + df['Nota Q12']) / 2

    # Imprimir mensaje de éxito con el número de filas procesadas correctamente
    print(f"Proceso de evaluación finalizado con éxito. Se evaluaron correctamente {filas_procesadas} de {df.shape[0]} respuestas.")

    return df


In [9]:
def generar_excel_y_pdf(df_resultado, nombre_sesion):
    """
    Genera un archivo Excel y un archivo PDF con las notas de los alumnos, y los guarda en el directorio de la sesión.
    El PDF no incluye la columna de emails.
    """
    # Ruta de salida
    output_dir = f"/workspace/reports/{nombre_sesion}"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Generar archivo Excel
    excel_path = os.path.join(output_dir, f"notas_{nombre_sesion}.xlsx")
    df_resultado[['Nombre completo', 'Nombre de usuario', 'Nota Q11', 'Nota Q12', 'Nota Final']].to_excel(excel_path, index=False)

    # Generar archivo PDF
    pdf = FPDF()
    pdf.add_page()

    # Configurar fuente y título del PDF
    pdf.set_font("Helvetica", 'B', 16)
    pdf.cell(200, 10, f"Notas de la sesión: {nombre_sesion}", new_x=XPos.LMARGIN, new_y=YPos.NEXT, align='C')

    # Espaciado
    pdf.ln(10)

    # Títulos de las columnas (sin email)
    pdf.set_font("Helvetica", 'B', 12)
    pdf.cell(100, 10, "Nombre del Alumno", border=1)
    pdf.cell(30, 10, "Nota Q11", border=1)
    pdf.cell(30, 10, "Nota Q12", border=1)
    pdf.cell(30, 10, "Nota Final", border=1)
    pdf.ln()

    # Datos de los alumnos (sin email)
    pdf.set_font("Helvetica", '', 12)
    for index, row in df_resultado.iterrows():
        pdf.cell(100, 10, row['Nombre completo'], border=1)
        pdf.cell(30, 10, str(row['Nota Q11']), border=1)
        pdf.cell(30, 10, str(row['Nota Q12']), border=1)
        pdf.cell(30, 10, str(row['Nota Final']), border=1)
        pdf.ln()

    # Guardar el PDF
    pdf_path = os.path.join(output_dir, f"notas_{nombre_sesion}.pdf")
    pdf.output(pdf_path)

    print(f"Archivos generados: {excel_path}, {pdf_path}")



In [10]:
def generar_informes(df_resultado, nombre_sesion, criterios_q11, criterios_q12):
    """
    Genera informes personalizados en formato Word para cada alumno en función de los resultados
    del DataFrame, los convierte a PDF, añade una nueva columna con la ruta al archivo PDF generado
    y agrega los criterios de evaluación formateados en el contexto.

    Parámetros:
    - df_resultado: DataFrame que contiene los resultados y comentarios de los alumnos.
    - nombre_sesion: Nombre de la sesión o curso, utilizado para nombrar el directorio y los archivos.
    - criterios_q11: Lista de criterios para la pregunta Q11.
    - criterios_q12: Lista de criterios para la pregunta Q12.

    Retorna:
    - df_resultado: DataFrame actualizado con una nueva columna que contiene la ruta a los archivos PDF generados.
    """

    # Crear el directorio de salida si no existe
    output_dir = f'/workspace/reports/{nombre_sesion}/alumnos'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)  # Crear el directorio si no existe

    informes_generados = 0  # Contador de informes generados

    # Crear una columna vacía en el DataFrame para almacenar las rutas de los PDFs
    df_resultado['Ruta PDF'] = None

    # Formatear los criterios para que se vean claramente en el documento
    criterios_q11_formateados = "\n".join([f"{linea}" if linea.startswith('1.') or linea.startswith('2.') or linea.startswith('3.') else f"    {linea}" for linea in criterios_q11])
    criterios_q12_formateados = "\n".join([f"{linea}" if linea.startswith('1.') or linea.startswith('2.') or linea.startswith('3.') else f"    {linea}" for linea in criterios_q12])

    # Iterar sobre cada fila del DataFrame para generar un informe por alumno usando tqdm para la barra de progreso
    for index, row in tqdm(df_resultado.iterrows(), total=len(df_resultado), desc="Generando informes", unit="informe"):
        try:
            # Cargar la plantilla de Word donde se insertarán los datos del alumno
            doc = DocxTemplate(TEMPLATE_PATH_ALUMNO)
        except FileNotFoundError:
            # Manejar el error si la plantilla no se encuentra
            print(f"Error: No se encontró la plantilla en la ruta especificada: {TEMPLATE_PATH_ALUMNO}")
            continue  # Saltar al siguiente alumno si no se encuentra la plantilla

        # Preparar el contexto con la información del alumno que se insertará en el informe
        context = {
            'nombre_alumno': row['Nombre completo'],
            'titulo_sesion': nombre_sesion,
            'pregunta_1': PREGUNTA_1,
            'respuesta_q11': row['Q11_Session content review 1'],
            'nota_q11': row['Nota Q11'],
            'comentario_q11': row['Comentario Q11'],
            'criterios_q11': criterios_q11_formateados,  # Incluir criterios formateados para Q11
            'pregunta_2': PREGUNTA_2,
            'respuesta_q12': row['Q12_Session content review 2'],
            'nota_q12': row['Nota Q12'],
            'comentario_q12': row['Comentario Q12'],
            'criterios_q12': criterios_q12_formateados,  # Incluir criterios formateados para Q12
            'nota_final': row['Nota Final']
        }

        # Intentar renderizar el documento con el contexto preparado
        try:
            doc.render(context)
        except Exception as e:
            # Manejar el error si hay problemas al renderizar el informe
            print(f"Error al renderizar el informe para {row['Nombre completo']}: {e}")
            continue  # Saltar al siguiente alumno si falla la renderización

        # Definir la ruta del archivo .docx de salida para el informe del alumno
        output_file = os.path.join(output_dir, f'{row["Nombre completo"].replace(" ", "_")}_{nombre_sesion}.docx')

        # Guardar el informe generado en formato Word
        try:
            doc.save(output_file)
        except OSError as e:
            # Manejar el error si no se puede guardar el archivo
            print(f"Error al guardar el informe para {row['Nombre completo']}: {e}")
            continue  # Saltar al siguiente alumno si falla al guardar

        # Convertir el archivo Word a PDF usando LibreOffice
        try:
            convert_to_pdf_with_libreoffice(output_file, output_dir)
            pdf_file = output_file.replace('.docx', '.pdf')  # Generar la ruta del archivo PDF

            # Añadir la ruta del PDF generado en la columna correspondiente del DataFrame
            df_resultado.at[index, 'Ruta PDF'] = pdf_file
            informes_generados += 1  # Incrementar el contador de informes generados
        except Exception as e:
            # Manejar el error si falla la conversión a PDF
            print(f"Error al convertir {output_file} a PDF: {e}")

    # Imprimir un mensaje al finalizar el proceso
    print(f"Proceso finalizado. Se generaron {informes_generados} informes.")
    
    # Retornar el DataFrame actualizado con las rutas a los archivos PDF generados
    return df_resultado


In [11]:
def convert_to_pdf_with_libreoffice(docx_path, output_dir):
    """Convierte un archivo .docx a PDF usando LibreOffice en modo de línea de comandos, solo muestra errores"""
    try:
        # Ejecutar el comando libreoffice para convertir el archivo a PDF sin mostrar los mensajes de éxito
        subprocess.run(
            ['libreoffice', '--headless', '--convert-to', 'pdf', docx_path, '--outdir', output_dir],
            check=True,
            stdout=subprocess.DEVNULL,  # Silenciar la salida estándar
            stderr=subprocess.PIPE  # Capturar errores estándar
        )
    except subprocess.CalledProcessError as e:
        print(f"Error al convertir el archivo .docx a PDF: {docx_path}\nDetalle del error: {e.stderr.decode()}")



In [12]:
def convertir_todos_los_docx_a_pdf(dir_reports):
    """Lee todos los archivos .docx en un directorio y los convierte a PDF secuencialmente, mostrando barra de progreso"""
    # Obtener todos los archivos .docx en el directorio
    docx_files = [os.path.join(dir_reports, file_name) for file_name in os.listdir(dir_reports) if file_name.endswith('.docx')]
    
    total_files = len(docx_files)  # Total de archivos .docx
    pdf_generated = 0  # Contador de PDFs generados correctamente

    # Mostrar barra de progreso usando tqdm
    for docx_path in tqdm(docx_files, desc="Convirtiendo archivos .docx a PDF"):
        try:
            convert_to_pdf_with_libreoffice(docx_path, dir_reports)
            pdf_generated += 1  # Incrementar el contador si se genera un PDF
        except Exception as e:
            print(f"Error durante la conversión de {docx_path}: {e}")

    # Imprimir un mensaje al finalizar con el número de archivos procesados
    print(f"La conversión de todos los archivos ha finalizado. Archivos .docx: {total_files}, Archivos PDF generados: {pdf_generated}")

In [13]:
def enviar_correo_con_adjunto(emails, nombre_sesion, pdf_path, nombre_alumno):
    """Envía un correo electrónico con el archivo PDF adjunto"""

    # Configurar el servidor SMTP (por ejemplo, Gmail)
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587
    smtp_user = os.getenv('SMTP_USER')  # Cargar desde .env
    smtp_password = os.getenv('SMTP_PASSWORD')  # Cargar desde .env

    # Calcular la fecha dos días a partir de hoy
    fecha_hoy_mas_dos = (datetime.today() + timedelta(days=2)).strftime('%Y-%m-%d')

    # Crear el mensaje
    asunto = f"Evaluation Report - Session {nombre_sesion}"
    mensaje = f"""Dear {nombre_alumno},

Attached you will find the report corresponding to the evaluation of the survey for the session "{nombre_sesion}". In the report, you will find the grade received, the criteria used for evaluation, as well as detailed comments on your responses.

If you believe the assigned grade does not accurately reflect your performance according to the provided criteria, you have the option to request a review. To do so, please send me an email to alfonso.santos@uam.es with the subject "Request for Evaluation Review - Session {nombre_sesion}" before 11:59 PM on {fecha_hoy_mas_dos}. In the email, you must clearly explain why the assigned grade is incorrect based on the evaluation criteria.

Please note that the review request must be based on a disagreement with the evaluation according to the provided criteria. The evaluation criteria cannot be changed, but the assigned grade is subject to review. Additionally, be aware that the review may result in either an improvement or a reduction in the grade.

I remain at your disposal for any further clarifications.

Sincerely,
Alfonso Santos
    """

    # Crear el objeto MIME para el mensaje
    msg = MIMEMultipart()
    msg['From'] = smtp_user
    msg['To'] = ', '.join(emails) if isinstance(emails, list) else emails
    msg['Subject'] = asunto

    # Adjuntar el texto del mensaje
    msg.attach(MIMEText(mensaje, 'plain'))

    # Adjuntar el archivo PDF
    try:
        with open(pdf_path, 'rb') as f:
            part = MIMEApplication(f.read(), _subtype='pdf')
            part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(pdf_path))
            msg.attach(part)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo PDF en la ruta: {pdf_path}")
        return

    # Enviar el correo
    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # Seguridad TLS
            server.login(smtp_user, smtp_password)
            server.sendmail(smtp_user, emails if isinstance(emails, list) else [emails], msg.as_string())
            print(f"Correo enviado con éxito a {emails}")
    except Exception as e:
        print(f"Error al enviar el correo: {e}")

In [14]:
def enviar_informes_por_correo(df_resultado, nombre_sesion):
    """
    Envía un correo electrónico a cada alumno con su archivo PDF adjunto.
    """
    emails_enviados = 0  # Contador para los correos enviados con éxito

    # Iterar sobre cada fila del DataFrame usando tqdm para la barra de progreso
    for index, row in tqdm(df_resultado.iterrows(), total=len(df_resultado), desc="Enviando correos", unit="correo"):
        try:
            nombre_alumno = row['Nombre completo']
            email_alumno = row['Nombre de usuario']  # Aquí obtenemos el correo de la columna 'Nombre de usuario'

            # Verificar que el PDF correspondiente existe
            pdf_path = row['Ruta PDF']
            if pdf_path:
                # Enviar el correo con el archivo PDF adjunto
                enviar_correo_con_adjunto(email_alumno, nombre_sesion, pdf_path, nombre_alumno)
                emails_enviados += 1  # Incrementar el contador de correos enviados con éxito
            else:
                print(f"Error: No se encontró el PDF para {nombre_alumno}. No se enviará un correo.")
        
        except Exception as e:
            # Imprimir el error exacto que ocurre dentro del bloque try
            print(f"Error al enviar el correo a {email_alumno} para {nombre_alumno}: {e}")

    # Imprimir el número de correos enviados con éxito al finalizar
    print(f"Proceso finalizado. Se enviaron {emails_enviados} correos con éxito.") 


## CÓDIGO EJECUCIÓN

ASIGNACIÓN DE VALORES PARA EJECUCIÓN. **MODIFICAR PARA CADA ENCUESTA**

In [15]:
# Constante para la ruta de la plantilla
TEMPLATE_PATH_ALUMNO = '/workspace/template/Evaluacion_Alumno_Template.docx'

# Ruta al archivo de criterios
CRITERIOS_PATH = '/workspace/src/criterios.txt'

# Ruta a los archivos de reports
DIR_REPORTS = '/workspace/reports'

# Datos de la sesión
NOMBRE_SESION = 'FinTech'
PREGUNTA_1 = 'What is a FinTech? Define the term and name at least three different types of fintech, describing each one in a sentence.'
PREGUNTA_2 = 'What have been the main contributions of FinTech companies to the financial sector? Mention at least three key contributions.'

# Ficherocon las respuestas de los alumnos
file_path_csv = "/workspace/data/FINTECH_SESSION_SURVEY.csv"


CARGO LAS CLAVES

In [16]:
# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave de API
openai_api_key = os.getenv("OPENAI_API_KEY")

# Verificar que se haya cargado correctamente
if openai_api_key:
    print("API key cargada correctamente.")
else:
    print("Error: No se pudo cargar la API key.")

API key cargada correctamente.


EVALÚA LAS ENCUESTAS, GENERA LOS REPORTS (Word y Pdf)

In [19]:
# Leer los criterios de evaluación
criterios_q11, criterios_q12 = leer_criterios(CRITERIOS_PATH)

# Leer las respuestas de los alumnos
df_encuestas = leer_encuestas(file_path_csv)

# Evaluar las respuestas en paralelo
df_resultado = evaluar_respuestas_df_paralelo(df_encuestas, criterios_q11, criterios_q12)

# Genera el PDF con las notas para subir a moodle y el excell para tener un respaldo
generar_excel_y_pdf(df_resultado, NOMBRE_SESION)

# Generar informes personalizados en formato Word y añadir la ruta del PDF en una nueva columna del DataFrame
df_resultado = generar_informes(df_resultado, NOMBRE_SESION, criterios_q11, criterios_q12)

Se leyeron 24 criterios para Q11 y 12 criterios para Q12 del archivo '/workspace/src/criterios.txt'.
Se han leído con éxito 38 registros del archivo CSV.
Proceso de evaluación finalizado con éxito. Se evaluaron correctamente 38 de 38 respuestas.
Archivos generados: /workspace/reports/FinTech/notas_FinTech.xlsx, /workspace/reports/FinTech/notas_FinTech.pdf


Generando informes: 100%|██████████| 38/38 [00:30<00:00,  1.24informe/s]

Proceso finalizado. Se generaron 38 informes.


Chequeo si para algún alumno no se ha generado el informe

In [18]:
# Verificar si hay algún alumno que no tiene PDF generado (columna 'Ruta PDF' vacía o nula)
alumnos_sin_informe = df_resultado[df_resultado['Ruta PDF'].isnull()]

# Si existen alumnos sin informe, se muestra la lista
if not alumnos_sin_informe.empty:
    print("Alumnos que no tienen informe generado:")
    print(alumnos_sin_informe[['Nombre completo']])
else:
    print("Todos los alumnos tienen informe generado.")

Todos los alumnos tienen informe generado.


**POR SI QUIERES HACER PRUEBAS DE ENVIAR EMAILS**

In [46]:
# df_resultado.loc[:, 'Nombre de usuario'] = 'alfonso.santos@uam.es'
# df_resultado = df_resultado.iloc[:3, :]
# df_resultado

 MANDA EMAILS CON LOS PDF  **OJO DESCOMENTAR LA LÍNEA DE CÓDIGO**

In [20]:
df_resultado

,Nombre completo,Nombre de usuario,Q11_Session content review 1,Q12_Session content review 2,Nota Q11,Comentario Q11,Nota Q12,Comentario Q12,Nota Final,Ruta PDF
0,Marcos Dominguez Valdes,marcos.dominguezv@estudiante.uam.es,Una fintech es una empresa que utiliza la tecn...,La principal contribucion de las fintech al se...,4.5,The student's definition of FinTech includes t...,0.0,The student's response does not adequately add...,2.25,/workspace/reports/FinTech/alumnos/Marcos_Domi...
1,Roberto García De Cabo,roberto.garciadecabo@estudiante.uam.es,Una FinTech es una empresa cuya base es la tec...,ayudar a cumplir los objetivos de las empresas,3.0,The definition of FinTech contains a few key e...,0.0,The student's response does not address any of...,1.50,/workspace/reports/FinTech/alumnos/Roberto_Gar...
2,Daniel Zubillaga Fernández,daniel.zubillaga@estudiante.uam.es,Se trata de una empresa que es tecnológica y s...,Una de ellas podrías ser cuando Microsoft Exce...,4.0,The student's response lacks a clear and compl...,0.0,The student's response does not address the qu...,2.00,/workspace/reports/FinTech/alumnos/Daniel_Zubi...
3,Mateo Revuelta Puente,mateo.revuelta@estudiante.uam.es,Una FinTech es toda empresa innovadora de bas...,Algunas de las contribuciones de las FinTech a...,7.0,The definition of FinTech provided includes es...,8.0,"The student provided a well-structured answer,...",7.50,/workspace/reports/FinTech/alumnos/Mateo_Revue...
4,Tingting Chen,tingting.chen@estudiante.uam.es,Fintech es una empresa con base tecnológica mu...,Permitir la democratización de las finanzas.Ag...,8.0,The student's response provides a coherent and...,7.0,The student identified three key contributions...,7.50,/workspace/reports/FinTech/alumnos/Tingting_Ch...
5,Andrea Rozada Suárez,andrea.rozada@estudiante.uam.es,Una fintech es una empresa con una avanzada ba...,Las aportaciones de las fintech al sector han ...,7.5,The definition of FinTech includes two key con...,9.0,The student provided a well-structured answer ...,8.25,/workspace/reports/FinTech/alumnos/Andrea_Roza...
6,Ana Belén Hornero Romera,ana.hornero@estudiante.uam.es,Fin-tech son empresas que defienden los derech...,Hacen que los asuntos financieros sean ahora m...,3.5,The definition of FinTech provided is incomple...,0.0,The student's response does not address the qu...,1.75,/workspace/reports/FinTech/alumnos/Ana_Belén_H...
7,Sonsoles Dorao Serrano,sonsoles.dorao@estudiante.uam.es,"empresa con base tecnologica, muy desarrollada...","han permitido transparecnia, seguridad, equida...",7.0,The student provided a definition of FinTech t...,3.0,The student's response includes three keywords...,5.00,/workspace/reports/FinTech/alumnos/Sonsoles_Do...
8,Izan Simarro Martin,izan.simarro@estudiante.uam.es,Una Fintech es una empresa con una base tecno...,Las tres principales contribuciones de las Fin...,8.5,The response provides a solid definition of Fi...,7.0,The student identifies three contributions of ...,7.75,/workspace/reports/FinTech/alumnos/Izan_Simarr...
9,Samuel Romero Arriero,samuel.romero@estudiante.uam.es,Una FinTech es una empresa basada principalmen...,Las principales contribuciones de las fintechs...,6.0,The student provides a definition of FinTech t...,0.0,The student's response does not adequately add...,3.00,/workspace/reports/FinTech/alumnos/Samuel_Rome...


In [21]:
# Enviar los correos con los informes adjuntos usando la nueva función enviar_informes_por_correo

enviar_informes_por_correo(df_resultado, NOMBRE_SESION)

Enviando correos:   3%|▎         | 1/38 [00:01<00:59,  1.62s/correo]

Correo enviado con éxito a marcos.dominguezv@estudiante.uam.es


Enviando correos:   5%|▌         | 2/38 [00:03<00:56,  1.58s/correo]

Correo enviado con éxito a roberto.garciadecabo@estudiante.uam.es


Enviando correos:   8%|▊         | 3/38 [00:04<00:57,  1.65s/correo]

...

Correo enviado con éxito a juan.gonzalob@estudiante.uam.es


Enviando correos:  97%|█████████▋| 37/38 [01:00<00:01,  1.62s/correo]

Correo enviado con éxito a marta-luciana.rosca@estudiante.uam.es


Enviando correos: 100%|██████████| 38/38 [01:01<00:00,  1.63s/correo]

Correo enviado con éxito a marco.pinnacarta@estudiante.uam.es
Proceso finalizado. Se enviaron 38 correos con éxito.
